In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2
from tensorflow import keras
from keras import layers

In [3]:
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)


def load_data(file):
    df = pd.read_csv(file)
    df.drop(columns="Unnamed: 0", inplace= True)
    df = df.set_index("Date")
    return df

def fit_model(model, cp, epochs, xt, yt, xv, yv):
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
    model.fit(xt, yt, validation_data=(xv, yv), epochs=epochs, callbacks=[cp])
    return model

def plot_val(model,Nsample, xv, yv):
    val_predictions = model.predict(xv).flatten()
    val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':yv})
    plt.plot(val_results['Val Predictions'][:Nsample])
    plt.plot(val_results['Actuals'][:Nsample])
    plt.title("Validation")
    plt.show()
    
def plot_test(model, Nsample, xtt, ytt):
    test_predictions = model.predict(xtt).flatten()
    test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':ytt})
    plt.plot(test_results['Test Predictions'][:Nsample])
    plt.plot(test_results['Actuals'][:Nsample])
    plt.title("Test")
    plt.show()
    
def plot_predictions1(model, X, y, start=0, end=100):
    prediction = model.predict(X).flatten()
    df = pd.DataFrame(data={'Predictions':prediction, 'Actuals':y})
    plt.plot(df['Predictions'][start:end])
    plt.plot(df['Actuals'][start:end])
    plt.title("Predictions")
    plt.show()
    mse_v = mse(y, prediction)
    mae_v = mae(y, prediction)
    r2_v = r2(y, prediction)

    return mse_v, mae_v, r2_v


def plot_train(modelname, Nsample, xt, yt):
    model = load_model(modelname)
    train_predictions = model.predict(xt).flatten()
    train_results = pd.DataFrame(data={'Train Predictions': train_predictions, 'Actuals':yt})
    plt.plot(train_results['Train Predictions'][:Nsample])
    plt.plot(train_results['Actuals'][:Nsample])
    plt.title("Train")
    plt.show()
    
def load_data_model(epochs):
    x_train = np.load(f"epochs{epochs}/x_train.npy")
    y_train = np.load(f"epochs{epochs}/y_train.npy")
    x_test  = np.load(f"epochs{epochs}/x_test.npy")
    y_test  = np.load(f"epochs{epochs}/y_test.npy")
    x_val   = np.load(f"epochs{epochs}/x_val.npy")
    y_val   = np.load(f"epochs{epochs}/y_val.npy")
    
    return x_train, y_train, x_test, y_test, x_val, y_val

In [4]:
epochs = 50
x_train, y_train, x_test, y_test, x_val, y_val = load_data_model(epochs)

In [5]:
x_train.shape

(230, 25, 1)

In [70]:
input_shape

(25, 1)

In [95]:
model = Sequential([
    LSTM(128, activation= "relu", input_shape= input_shape, return_sequences= True),
    LSTM(64, activation= "relu", return_sequences= False),
    RepeatVector(25),
    LSTM(64, activation= "relu", return_sequences= True),
    LSTM(64, activation= "relu", return_sequences= True),
    TimeDistributed(Dense(1))
])

model.compile(optimizer = "adam", loss="mse")
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_56 (LSTM)              (None, 25, 128)           66560     
                                                                 
 lstm_57 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 25, 64)           0         
 or)                                                             
                                                                 
 lstm_58 (LSTM)              (None, 25, 64)            33024     
                                                                 
 lstm_59 (LSTM)              (None, 25, 64)            33024     
                                                                 
 time_distributed_1 (TimeDis  (None, 25, 1)            65        
 tributed)                                           

In [ ]:
model = Sequential([
  Input(shape=input_shape),
  Conv1D(filters=64, kernel_size=3, activation="relu"),
  MaxPooling1D(pool_size=2),
  LSTM(units=8, return_sequences=True),  # Encode to a single vector
  LSTM(units=8, return_sequences=True),  # Forecast for multiple steps
  UpSampling1D(size=1),  # Upsample for each predicted step
  Conv1D(filters=64, kernel_size=3, activation="relu"),
  Dense(5)  # Single output for each predicted step
])

model.compile(optimizer="adam", loss="mse")

model.summary()

In [96]:
model.fit(x_train, y_train, epochs= 20)

Epoch 1/20
8/8 [==============================] - 10s 78ms/step - loss: 6627.2051
Epoch 2/20
8/8 [==============================] - 1s 71ms/step - loss: 3146.8198
Epoch 3/20
8/8 [==============================] - 1s 65ms/step - loss: 4923.1382
Epoch 4/20
8/8 [==============================] - 1s 65ms/step - loss: 20980.5957
Epoch 5/20
8/8 [==============================] - 1s 76ms/step - loss: 22685.3516
Epoch 6/20
8/8 [==============================] - 1s 65ms/step - loss: 46190.1211
Epoch 7/20
8/8 [==============================] - 1s 65ms/step - loss: 43111.8008
Epoch 8/20
8/8 [==============================] - 1s 67ms/step - loss: 21073.4043
Epoch 9/20
8/8 [==============================] - 1s 66ms/step - loss: 15792.8057
Epoch 10/20
8/8 [==============================] - 1s 65ms/step - loss: 13389.6787
Epoch 11/20
8/8 [==============================] - 1s 64ms/step - loss: 18212.5762
Epoch 12/20
8/8 [==============================] - 1s 69ms/step - loss: 22208.9082
Epoch 13/20
8/8

In [105]:
predicted = model.predict(x_test)

3/3 [==============================] - 0s 18ms/step


In [106]:
predicted[:1]

array([[[ -29.92317 ],
        [ -45.835613],
        [  -4.991145],
        [ -96.280235],
        [-160.75795 ],
        [-210.21872 ],
        [-189.86017 ],
        [-211.3813  ],
        [-332.15433 ],
        [-306.15097 ],
        [-269.42383 ],
        [-256.9978  ],
        [-253.49228 ],
        [-216.4922  ],
        [-242.8556  ],
        [ -93.898735],
        [ -32.593437],
        [-174.74234 ],
        [ -77.39383 ],
        [-301.6774  ],
        [  71.98162 ],
        [ 137.42932 ],
        [  68.71011 ],
        [  56.45937 ],
        [  15.82942 ]]], dtype=float32)

In [107]:
predicted.shape

(72, 25, 1)

In [112]:
scaler = StandardScaler()
scaler = scaler.fit(x_train)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [110]:
predicted[0].mean()

-126.26845

In [104]:
predict = []
 
for a in predicted:
    mean =

(1800,)

In [102]:
y_test.shape

(72,)

In [100]:
plot_predictions1(model, x_test, y_test)

3/3 [==============================] - 0s 21ms/step


ValueError: All arrays must be of the same length

In [40]:
def autoencoder_cnn(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv1D(32, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling1D(2, padding='same')(conv1)
    conv2 = Conv1D(16, 3, activation='relu', padding='same')(pool1)
    encoded = MaxPooling1D(2, padding='same')(conv2)

    # Decoder
    conv3 = Conv1D(16, 3, activation='relu', padding='same')(encoded)
    up1 = UpSampling1D(2)(conv3)
    conv4 = Conv1D(32, 3, activation='relu', padding='same')(up1)
    up2 = UpSampling1D(2)(conv4)
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(up2)

    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [41]:
def forecasting_model(autoencoder):
    # Create a model that takes input from the autoencoder's encoder part
    forecasting_input = autoencoder.layers[1].output
    x = Conv1D(32, 3, activation='relu', padding='same')(forecasting_input)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(16, 3, activation='relu', padding='same')(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='linear')(x)

    forecasting_model = Model(inputs=autoencoder.input, outputs=output)
    forecasting_model.compile(optimizer='adam', loss='mse')
    return forecasting_model

In [42]:
input_shape = (25, 1)
autoencoder = autoencoder_cnn(input_shape)
forecasting_model = forecasting_model(autoencoder)
autoencoder.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 25, 1)]           0         
                                                                 
 conv1d_34 (Conv1D)          (None, 25, 32)            128       
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 13, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_35 (Conv1D)          (None, 13, 16)            1552      
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 7, 16)            0         
 g1D)                                                            
                                                                 
 conv1d_36 (Conv1D)          (None, 7, 16)             784

In [43]:
forecasting_model.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 25, 1)]           0         
                                                                 
 conv1d_34 (Conv1D)          (None, 25, 32)            128       
                                                                 
 conv1d_39 (Conv1D)          (None, 25, 32)            3104      
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 13, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_40 (Conv1D)          (None, 13, 16)            1552      
                                                                 
 max_pooling1d_22 (MaxPoolin  (None, 7, 16)            0         
 g1D)                                                     

In [44]:
autoencoder.fit(x_train, x_train, epochs=10)
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[4].output)
encoded_X_train = encoder.predict(x_train)

Epoch 1/10


ValueError: in user code:

    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/home/RHHLab/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = 

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_seq_length = 28 
x_train_padded = pad_sequences(x_train, maxlen=max_seq_length, padding='post', truncating='post')

In [50]:
x_train_resized = x_train_padded[:, :25, :] 